In [ ]:
##Download Shopify dataset

In [1]:
!gsutil cp gs://cs327e-open-access/shopify.zip .

Copying gs://cs327e-open-access/shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [2]:
!unzip shopify.zip

Archive:  shopify.zip
   creating: shopify/
  inflating: __MACOSX/._shopify      
  inflating: shopify/apps_categories.csv  
  inflating: __MACOSX/shopify/._apps_categories.csv  
  inflating: shopify/categories.csv  
  inflating: __MACOSX/shopify/._categories.csv  
  inflating: shopify/reviews.csv     
  inflating: __MACOSX/shopify/._reviews.csv  
  inflating: shopify/.DS_Store       
  inflating: __MACOSX/shopify/._.DS_Store  
  inflating: shopify/key_benefits.csv  
  inflating: __MACOSX/shopify/._key_benefits.csv  
  inflating: shopify/pricing_plan_features.csv  
  inflating: __MACOSX/shopify/._pricing_plan_features.csv  
  inflating: shopify/pricing_plans.csv  
  inflating: __MACOSX/shopify/._pricing_plans.csv  
  inflating: shopify/apps.csv        
  inflating: __MACOSX/shopify/._apps.csv  


In [ ]:
##Create Firestore collections and subcollections

In [28]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()

In [29]:
# access pattern 1: get apps by Category

batch = db.batch()

df = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n')
categories_rows = df.values.tolist()

df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

df  = pd.read_csv('shopify/apps_categories.csv', sep=',',  header=0, lineterminator='\n')
apps_categories_rows = df.values.tolist()

record_count = 0

for categories_row in categories_rows:
    
    categories_record = {}
    categories_record['id'] = categories_row[0]
    categories_record['title'] = categories_row[1]
    
    categories_ref = db.collection('categories').document(categories_row[1])
    
    batch.set(categories_ref, categories_record)
    
    for apps_categories_row in apps_categories_rows:
        
        if categories_row[0] == apps_categories_row[1]:
            
            for apps_row in apps_rows:
                app_record = {}
                if apps_categories_row[0] == apps_row[0]:
                    app_record['id'] = apps_row[0]
                    app_record['url'] = apps_row[1]
                    app_record['title'] = apps_row[2]
                    app_record['developer'] = apps_row[3]
                    app_record['developer_link'] = apps_row[4]
                    app_record['icon'] = apps_row[5]
                    app_record['rating'] = apps_row[6]
                    app_record['review_count'] = apps_row[7]
            
            
                    apps_ref = categories_ref.collection('apps').document(apps_row[0].replace("/", ""))
                    batch.set(apps_ref, app_record)
                    record_count += 1
                    if record_count==399:
                        batch.commit()
                        record_count=0
    
batch.commit()

[update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_time {
   seconds: 1634864649
   nanos: 725450000
 },
 update_

In [37]:
# access pattern 2-4: get apps with highest review count, as well as pricing plan details and key benefits by app

batch = db.batch()

df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

df = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
p_plan_rows = df.values.tolist()

df = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
kb_rows = df.values.tolist()

df = pd.read_csv('shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
ppf_rows = df.values.tolist()

record_count = 0

for apps_row in apps_rows:
    apps_record = {}
    apps_record['id'] = apps_row[0]
    apps_record['url'] = apps_row[1]
    apps_record['title'] = apps_row[2]
    apps_record['developer'] = apps_row[3]
    apps_record['developer_link'] = apps_row[4]
    apps_record['icon'] = apps_row[5]
    apps_record['rating'] = apps_row[6]
    apps_record['review_count'] = apps_row[7]
    
    apps_ref = db.collection('apps').document(apps_row[0])
    batch.set(apps_ref, apps_record)
    if record_count == 399:
        batch.commit()
        record_count = 0   
    record_count += 1
    
    for p_plan_row in p_plan_rows:
        if apps_row[0] == p_plan_row[1]:
            p_plan_record = {}
            p_plan_record['id'] = p_plan_row[0]
            p_plan_record['title'] = p_plan_row[2]
            p_plan_record['price'] = p_plan_row[3]
            p_plan_record['features'] = []
            
            for ppf_row in ppf_rows:
                if p_plan_row[0] == ppf_row[1]:
                    p_plan_record['features'].append(ppf_row[1])
            
            p_plan_ref = apps_ref.collection('pricing plans').document(p_plan_row[0].replace("/", ""))
            batch.set(p_plan_ref, p_plan_record)
            
    
    for kb_row in kb_rows:
        if apps_row[0] == kb_row[0]:
            kb_record = {}
            kb_record['title'] = kb_row[1]
            kb_record['description'] = kb_row[2]
            
            kb_ref = apps_ref.collection('key benefits').document(kb_row[1].replace("/", ""))
            batch.set(kb_ref, kb_record)
            
        
        
    batch.commit()
    

In [ ]:
# get document count for each collection and subcollection

In [11]:
doc_count = db.collection('categories')
count = list(doc_count.get())
print('Document count for <CATEGORIES>: {}'.format(len(count)))

Document count for <CATEGORIES>: 12


In [12]:
doc_count = db.collection_group('apps')
count = list(doc_count.get())
print('Document count for subcollection <APPS>: {}'.format(len(count)))

Document count for subcollection <APPS>: 8930


In [13]:
doc_count = db.collection('apps')
count = list(doc_count.get())
print('Document count for <APPS>: {}'.format(len(count)))

Document count for <APPS>: 3547


In [14]:
doc_count = db.collection_group('pricing plans')
count = list(doc_count.get())
print('Document count for subcollection <PRICING PLANS>: {}'.format(len(count)))

Document count for subcollection <PRICING PLANS>: 6275


In [15]:
doc_count = db.collection_group('key benefits')
count = list(doc_count.get())
print('Document count for subcollection <KEY BENEFITS>: {}'.format(len(count)))

Document count for subcollection <KEY BENEFITS>: 9541


In [44]:
#Query 1: List top 10 Producitivity apps by rating in descending order

categories_ref = db.collection('categories')
query = categories_ref.where('title', '==', 'Productivity')
results = query.stream()

for result in results:
    app_results = categories_ref.document('Productivity').collection('apps').order_by('rating', direction=firestore.Query.DESCENDING).limit(10).stream()
    for app_result in app_results:
        ar_id = app_result.id
        app_result = app_result.to_dict()
        del app_result['url']
        del app_result['developer_link']
        del app_result['icon']
        print(ar_id, '=>', app_result)
  

fe5ae45c-379c-42bf-be7d-b5f6e15fc13c => {'title': 'Customer Tags', 'review_count': 21, 'rating': 5.0, 'id': 'fe5ae45c-379c-42bf-be7d-b5f6e15fc13c', 'developer': 'Union Works Apps'}
f99bb1e3-f326-4f10-8901-491652e9809b => {'review_count': 68, 'title': 'Order Tagger', 'rating': 5.0, 'developer': 'Union Works Apps', 'id': 'f99bb1e3-f326-4f10-8901-491652e9809b'}
f864e3bd-da0e-41dc-be65-984325331475 => {'developer': 'SilkRoad', 'title': 'SilkRoad ‑ Facebook Auto Ads', 'review_count': 2, 'id': 'f864e3bd-da0e-41dc-be65-984325331475', 'rating': 5.0}
f5344e64-9cda-4d97-b198-2aaeb5170518 => {'developer': 'Modd Apps Inc.', 'id': 'f5344e64-9cda-4d97-b198-2aaeb5170518', 'rating': 5.0, 'title': 'Xporter Data Export Tool', 'review_count': 223}
ed77a32d-0fa3-458b-b639-e01ea7b78ec0 => {'review_count': 6, 'developer': 'ShopFox', 'title': 'FraudBlock Fraud Prevention', 'rating': 5.0, 'id': 'ed77a32d-0fa3-458b-b639-e01ea7b78ec0'}
e0e231d6-4988-4a8c-ad84-cf18d0f38738 => {'rating': 5.0, 'title': 'Automation

In [45]:
#Query 2: list the 10 apps with the highest number of reviews

In [50]:
apps_ref = db.collection('apps')
query = apps_ref.order_by('review_count',direction=firestore.Query.DESCENDING).limit(10)
app_results = query.stream()

for app_result in app_results:
        ar_id = app_result.id
        app_result = app_result.to_dict()
        del app_result['url']
        del app_result['developer_link']
        del app_result['icon']
        print(ar_id, '=>', app_result)

d9f142ee-b141-4dc4-9353-173db61d2eb0 => {'id': 'd9f142ee-b141-4dc4-9353-173db61d2eb0', 'title': 'Privy ‑ Exit Pop Ups & Email', 'developer': 'Privy', 'review_count': 23078, 'rating': 4.7}
78ea0810-c008-4a4e-a82f-de0c790e3286 => {'review_count': 8737, 'developer': 'Hextom', 'id': '78ea0810-c008-4a4e-a82f-de0c790e3286', 'rating': 4.9, 'title': 'Free Shipping Bar'}
b88488b0-9912-44d3-b736-224c36f09d95 => {'id': 'b88488b0-9912-44d3-b736-224c36f09d95', 'developer': 'CartKit', 'title': 'Sales Pop ‑ Popup Notification', 'review_count': 6905, 'rating': 4.8}
e528a60e-94f8-4e92-80e2-5bc6013b8283 => {'title': 'BEST Currency Converter', 'id': 'e528a60e-94f8-4e92-80e2-5bc6013b8283', 'review_count': 5986, 'rating': 4.8, 'developer': 'Grizzly Apps'}
be2640c4-01b5-4d52-9f68-cae8c0734d0d => {'developer': 'Recart', 'title': 'Recart FB Messenger Marketing', 'review_count': 5596, 'id': 'be2640c4-01b5-4d52-9f68-cae8c0734d0d', 'rating': 4.8}
70bff9e0-4316-4cc6-84ce-92fcd1bc6925 => {'developer': 'Booster App